# Stress analysis for each element 

In [1]:
name = 'yannis'

# Import modules 

In [2]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.panels import *
from formulas.helpers import *
from formulas.abd_matrix import *
from formulas.Strength import * 

# Import needed files 

## Import panel stresses

In [3]:
PlyStressesX = pd.read_csv(f'../data/{name}/compositePanelXX.csv')
PlyStressesX = PlyStressesX.drop(columns=['FileID', 'Step'])
PlyStressesY = pd.read_csv(f'../data/{name}/compositePanelYY.csv')
PlyStressesY = PlyStressesY.drop(columns=['FileID', 'Step'])
PlyStressesXY = pd.read_csv(f'../data/{name}/compositePanelXY.csv')
PlyStressesXY = PlyStressesXY.drop(columns=['FileID', 'Step'])
PlyStressesX

,Elements,Loadcase,Layer,Composite Stresses:Normal X Stress
0,1,1,Ply 1,-35.419704
1,2,1,Ply 1,25.747299
2,3,1,Ply 1,2.299717
3,4,1,Ply 1,-22.946745
4,5,1,Ply 1,14.230029
...,...,...,...,...
1435,26,3,Ply 9,15.286470
1436,27,3,Ply 9,-46.393162
1437,28,3,Ply 9,66.984344
1438,29,3,Ply 9,18.347198


In [4]:
PlyStressesPanel=pd.merge(PlyStressesX,PlyStressesY, on=['Elements', 'Layer', 'Loadcase'])
PlyStressesPanel = pd.merge(PlyStressesPanel, PlyStressesXY, on=['Elements', 'Layer', 'Loadcase'])
PlyStressesPanel = PlyStressesPanel.rename(columns={'Composite Stresses:Normal X Stress':'Normal_1', 'Composite Stresses:Normal Y Stress':'Normal_2', 'Composite Stresses:Shear XY Stress':'Shear_12' })
PlyStressesPanel

,Elements,Loadcase,Layer,Normal_1,Normal_2,Shear_12
0,1,1,Ply 1,-35.419704,-10.641421,11.146913
1,2,1,Ply 1,25.747299,-3.627014,4.989726
2,3,1,Ply 1,2.299717,-11.597987,12.994295
3,4,1,Ply 1,-22.946745,-10.609735,8.319627
4,5,1,Ply 1,14.230029,-4.767473,4.595075
...,...,...,...,...,...,...
1435,26,3,Ply 9,15.286470,-2.278381,-12.591271
1436,27,3,Ply 9,-46.393162,-2.525533,-14.392484
1437,28,3,Ply 9,66.984344,-1.739914,-14.648998
1438,29,3,Ply 9,18.347198,-3.127240,-11.663470


## Import stringer plies 

In [5]:
StringerStrains = pd.read_csv(f'../data/{name}/stringer_strain.csv')
StringerStrains = StringerStrains.drop(columns=['FileID', 'Step'])
StringerStrains = StringerStrains.rename(columns={'Element Strains (1D):CBAR/CBEAM Axial Strain':'strainX'})
StringerStrains

,Elements,Loadcase,strainX
0,40,1,-0.001215
1,41,1,-0.001502
2,42,1,-0.001270
3,46,1,-0.001138
4,47,1,-0.001586
5,48,1,-0.001434
6,52,1,-0.001434
7,53,1,-0.001586
8,54,1,-0.001138
9,58,1,-0.001270


# Constants for analysis 

In [6]:
# r = perpendicular 
# p = parallel
# c = compression 
# t = tension 
# Resistance values 

R_p_t = 3050
R_p_c = 1500
R_r_c = 50
R_r_t = 300 
R_rp = 100

# p values 
# Perpendicular, parallel
p_rp_c = 0.25
p_rp_t = 0.25

# Perpendicular, perpendicular
p_rr_c = 0.25
p_rr_t = 0.25

#Ply orientation in flange 
StringerFlange=[45,45,-45,-45,0,0,90,90,90,90,0,0,-45,-45,45,45]

# Personal data Provider 

In [7]:
personal_data = personal_data_provider(name)
E_11 = personal_data[0]
E_22 = personal_data[1]
G_12 = personal_data[2]
nu_12 = personal_data[3]

# Calculate RF and mode for panels

## Add the ultimate load factor 

In [8]:
PlyStressesPanel['Normal_1'] = PlyStressesPanel['Normal_1'] * 1.5
PlyStressesPanel['Normal_2'] = PlyStressesPanel['Normal_2'] * 1.5
PlyStressesPanel['Shear_12'] = PlyStressesPanel['Shear_12'] * 1.5

In [9]:
PlyStressesPanel[['Mode', 'RF_IFF', 'RF_FF']] = PlyStressesPanel.apply(strength, R_p_t=R_p_t, R_p_c=R_p_c, R_r_c=R_r_c, R_r_t=R_r_t, 
                                                    R_rp=R_rp, p_rp_c=p_rp_c, p_rp_t=p_rp_t, p_rr_c=p_rr_c, p_rr_t=p_rr_t,
                                                    axis=1, result_type='expand')
PlyStressesPanel

,Elements,Loadcase,Layer,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF
0,1,1,Ply 1,-53.129557,-15.962132,16.720369,C,3.000711,28.232873
1,2,1,Ply 1,38.620949,-5.440521,7.484588,C,8.543523,78.972684
2,3,1,Ply 1,3.449576,-17.396981,19.491443,C,2.736651,884.166690
3,4,1,Ply 1,-34.420117,-15.914602,12.479440,C,3.066350,43.579166
4,5,1,Ply 1,21.345044,-7.151210,6.892613,C,6.741320,142.890314
...,...,...,...,...,...,...,...,...,...
1435,26,3,Ply 9,22.929706,-3.417571,-18.886907,C,6.585321,133.015227
1436,27,3,Ply 9,-69.589743,-3.788300,-21.588726,C,5.740868,21.554901
1437,28,3,Ply 9,100.476517,-2.609871,-21.973497,B,4.688076,30.355352
1438,29,3,Ply 9,27.520798,-4.690860,-17.495205,C,6.848480,110.825276


## Calculate RF_strength 

In [10]:
PlyStressesPanel['RF_strength'] = PlyStressesPanel[['RF_IFF', 'RF_FF']].min(axis=1)
PlyStressesPanel.head(5)

,Elements,Loadcase,Layer,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF,RF_strength
0,1,1,Ply 1,-53.129557,-15.962132,16.720369,C,3.000711,28.232873,3.000711
1,2,1,Ply 1,38.620949,-5.440521,7.484588,C,8.543523,78.972684,8.543523
2,3,1,Ply 1,3.449576,-17.396981,19.491443,C,2.736651,884.166690,2.736651
3,4,1,Ply 1,-34.420117,-15.914602,12.479440,C,3.066350,43.579166,3.066350
4,5,1,Ply 1,21.345044,-7.151210,6.892613,C,6.741320,142.890314,6.741320


## Convert to output for panels 

In [11]:
# We can drop the stresses 
PlyStressesPanel = PlyStressesPanel.drop(columns=['Normal_1', 'Normal_2', 'Shear_12'])

#Reorder plies 
PlyStressesPanel['Layer_num'] = PlyStressesPanel['Layer'].str.extract('(\d+)').astype(int)
PlyStressesPanel = PlyStressesPanel.sort_values(by='Layer_num')
PlyStressesPanel = PlyStressesPanel.drop(columns=['Layer'])
PlyStressesPanel = PlyStressesPanel.rename(columns={'Layer_num':'Layer'})

# Resort columns to desired order
PlyStressesPanel = PlyStressesPanel[['Elements', 'Layer', 'Loadcase', 'RF_FF', 'RF_IFF', 'Mode', 'RF_strength']]

# We need to analyse element 8 
PlyStressesPanel = PlyStressesPanel[PlyStressesPanel['Elements'] == 8]
#Now we separate the loadcases 
PlyStressesPanelLC1 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 1]
PlyStressesPanelLC2 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 2]
PlyStressesPanelLC3 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 3]
PlyStressesPanelLC3

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Yanni\AppData\Local\Temp\ipykernel_7924\2671095803.py:5: SyntaxWarning: invalid escape sequence '\d'
  PlyStressesPanel['Layer_num'] = PlyStressesPanel['Layer'].str.extract('(\d+)').astype(int)


,Elements,Layer,Loadcase,RF_FF,RF_IFF,Mode,RF_strength
967,8,1,3,12.998722,3.411502,C,3.411502
1207,8,2,3,13.061302,3.429606,C,3.429606
1237,8,3,3,5.744113,17.658631,A,5.744113
1267,8,4,3,5.774447,17.515984,A,5.774447
1297,8,5,3,30.735320,5.201876,A,5.201876
1327,8,6,3,30.295791,5.224591,A,5.224591
1357,8,7,3,135.776304,6.546179,C,6.546179
1387,8,8,3,130.235819,6.592437,C,6.592437
1417,8,9,3,125.129777,6.637743,C,6.637743
997,8,10,3,120.408999,6.682089,C,6.682089


## Rename Loadcases 

In [12]:
# Loadcase 1
PlyStressesPanelLC1 = PlyStressesPanelLC1.rename(columns={'RF_FF':'R_FF_LC1', 'RF_IFF':'R_IFF_LC1', 'Mode':'Mode_LC1', 'RF_strength':'RF_strength_LC1'})
PlyStressesPanelLC1 = PlyStressesPanelLC1.reset_index()
PlyStressesPanelLC1 = PlyStressesPanelLC1.drop(columns=['index', 'Loadcase'])

# Loadcase 2
PlyStressesPanelLC2 = PlyStressesPanelLC2.rename(columns={'RF_FF':'R_FF_LC2', 'RF_IFF':'R_IFF_LC2', 'Mode':'Mode_LC2', 'RF_strength':'RF_strength_LC2'})
PlyStressesPanelLC2 = PlyStressesPanelLC2.reset_index()
PlyStressesPanelLC2 = PlyStressesPanelLC2.drop(columns=['index', 'Loadcase'])

#Loadcase 3 
PlyStressesPanelLC3 = PlyStressesPanelLC3.rename(columns={'RF_FF':'R_FF_LC3', 'RF_IFF':'R_IFF_LC3', 'Mode':'Mode_LC3', 'RF_strength':'RF_strength_LC3'})
PlyStressesPanelLC3 = PlyStressesPanelLC3.reset_index()
PlyStressesPanelLC3 = PlyStressesPanelLC3.drop(columns=['index', 'Loadcase'])
PlyStressesPanelLC1

,Elements,Layer,R_FF_LC1,R_IFF_LC1,Mode_LC1,RF_strength_LC1
0,8,1,128.968032,6.810143,C,6.810143
1,8,2,132.708406,6.764428,C,6.764428
2,8,3,15.062606,12.715390,B,12.715390
3,8,4,15.020256,12.480258,B,12.480258
4,8,5,10.259166,19.362779,A,10.259166
5,8,6,10.083506,19.360291,A,10.083506
6,8,7,43.418012,5.004355,C,5.004355
7,8,8,42.451271,4.927935,C,4.927935
8,8,9,41.526642,4.853784,C,4.853784
9,8,10,40.641436,4.781805,C,4.781805


## Combine Loadcases 

In [13]:
PlyStressesPanel = PlyStressesPanelLC1.merge(PlyStressesPanelLC2, on=['Elements', 'Layer'], suffixes=('_LC1', '_LC2'))
PlyStressesPanel = PlyStressesPanel.merge(PlyStressesPanelLC3, on=['Elements', 'Layer'])
PlyStressesPanel

,Elements,Layer,R_FF_LC1,R_IFF_LC1,Mode_LC1,RF_strength_LC1,R_FF_LC2,R_IFF_LC2,Mode_LC2,RF_strength_LC2,R_FF_LC3,R_IFF_LC3,Mode_LC3,RF_strength_LC3
0,8,1,128.968032,6.810143,C,6.810143,613.126955,5.591992,C,5.591992,12.998722,3.411502,C,3.411502
1,8,2,132.708406,6.764428,C,6.764428,777.897815,5.538677,C,5.538677,13.061302,3.429606,C,3.429606
2,8,3,15.062606,12.715390,B,12.715390,12.649242,16.580488,C,12.649242,5.744113,17.658631,A,5.744113
3,8,4,15.020256,12.480258,B,12.480258,12.565487,16.101392,C,12.565487,5.774447,17.515984,A,5.774447
4,8,5,10.259166,19.362779,A,10.259166,11.556460,18.880624,C,11.556460,30.735320,5.201876,A,5.201876
5,8,6,10.083506,19.360291,A,10.083506,11.422447,18.543368,C,11.422447,30.295791,5.224591,A,5.224591
6,8,7,43.418012,5.004355,C,5.004355,312.827604,4.999687,C,4.999687,135.776304,6.546179,C,6.546179
7,8,8,42.451271,4.927935,C,4.927935,323.486917,4.939296,C,4.939296,130.235819,6.592437,C,6.592437
8,8,9,41.526642,4.853784,C,4.853784,334.898294,4.880328,C,4.880328,125.129777,6.637743,C,6.637743
9,8,10,40.641436,4.781805,C,4.781805,347.144180,4.822735,C,4.822735,120.408999,6.682089,C,6.682089


# Output panel 

In [14]:
PlyStressesPanel.to_excel(f'../data/{name}/output/panelStrength.xlsx')

# Calculate stringer strength 

In [15]:
StringerStrains.head(5)

,Elements,Loadcase,strainX
0,40,1,-0.001215
1,41,1,-0.001502
2,42,1,-0.001270
3,46,1,-0.001138
4,47,1,-0.001586


## Expand the dataframe 

In [16]:
# Create a DataFrame with PlyNumber 1-16
ply_numbers = pd.DataFrame({'PlyNumber': np.arange(1, 17),
                            'plyTheta': StringerFlange})

# Cross join to expand each element row to 16 plies
StringerStrains = StringerStrains.merge(ply_numbers, how='cross')
#StringerStrains = StringerStrains.drop(columns=['PlyNumber_x'])
StringerStrains.head(33)

,Elements,Loadcase,strainX,PlyNumber,plyTheta
0,40,1,-0.001215,1,45
1,40,1,-0.001215,2,45
2,40,1,-0.001215,3,-45
3,40,1,-0.001215,4,-45
4,40,1,-0.001215,5,0
5,40,1,-0.001215,6,0
6,40,1,-0.001215,7,90
7,40,1,-0.001215,8,90
8,40,1,-0.001215,9,90
9,40,1,-0.001215,10,90


In [17]:
StringerStrains[['Normal_1', 'Normal_2', 'Shear_12']] = StringerStrains.apply(
    calculateMatStress, EModulus1=E_11, EModulus2=E_22, ShearModulus=G_12, axis=1,
    result_type='expand'
)
StringerStrains

,Elements,Loadcase,strainX,PlyNumber,plyTheta,Normal_1,Normal_2,Shear_12
0,40,1,-0.001215,1,45,-74.950604,-7.478190,5.575598
1,40,1,-0.001215,2,45,-74.950604,-7.478190,5.575598
2,40,1,-0.001215,3,-45,-74.950604,-7.478190,-5.575598
3,40,1,-0.001215,4,-45,-74.950604,-7.478190,-5.575598
4,40,1,-0.001215,5,0,-146.190227,-3.710982,0.000000
...,...,...,...,...,...,...,...,...
571,60,3,-0.000346,12,0,-41.672474,-1.057839,0.000000
572,60,3,-0.000346,13,-45,-21.365157,-2.131707,-1.589361
573,60,3,-0.000346,14,-45,-21.365157,-2.131707,-1.589361
574,60,3,-0.000346,15,45,-21.365157,-2.131707,1.589361


## Now calculate the FF and the IFF 

### Add ultimate load factor 

In [18]:
StringerStrains['Normal_1'] = StringerStrains['Normal_1'] * 1.5
StringerStrains['Normal_2'] = StringerStrains['Normal_2'] * 1.5
StringerStrains['Shear_12'] = StringerStrains['Shear_12'] * 1.5

In [19]:
StringerStrains[['Mode', 'RF_IFF', 'RF_FF']] = StringerStrains.apply(strength, R_p_t=R_p_t, R_p_c=R_p_c, R_r_c=R_r_c, R_r_t=R_r_t, 
                                                    R_rp=R_rp, p_rp_c=p_rp_c, p_rp_t=p_rp_t, p_rr_c=p_rr_c, p_rr_t=p_rr_t,
                                                    axis=1, result_type='expand')
StringerStrains

,Elements,Loadcase,strainX,PlyNumber,plyTheta,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF
0,40,1,-0.001215,1,45,-112.425907,-11.217285,8.363398,C,4.360449,13.342121
1,40,1,-0.001215,2,45,-112.425907,-11.217285,8.363398,C,4.360449,13.342121
2,40,1,-0.001215,3,-45,-112.425907,-11.217285,-8.363398,C,4.360449,13.342121
3,40,1,-0.001215,4,-45,-112.425907,-11.217285,-8.363398,C,4.360449,13.342121
4,40,1,-0.001215,5,0,-219.285341,-5.566472,0.000000,C,8.982349,6.840403
...,...,...,...,...,...,...,...,...,...,...,...
571,60,3,-0.000346,12,0,-62.508712,-1.586759,0.000000,C,31.510769,23.996655
572,60,3,-0.000346,13,-45,-32.047735,-3.197560,-2.384041,C,15.296789,46.805179
573,60,3,-0.000346,14,-45,-32.047735,-3.197560,-2.384041,C,15.296789,46.805179
574,60,3,-0.000346,15,45,-32.047735,-3.197560,2.384041,C,15.296789,46.805179


In [20]:
StringerStrains['RF_strength'] = StringerStrains[['RF_IFF', 'RF_FF']].min(axis=1)
StringerStrains.head(5)

,Elements,Loadcase,strainX,PlyNumber,plyTheta,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF,RF_strength
0,40,1,-0.001215,1,45,-112.425907,-11.217285,8.363398,C,4.360449,13.342121,4.360449
1,40,1,-0.001215,2,45,-112.425907,-11.217285,8.363398,C,4.360449,13.342121,4.360449
2,40,1,-0.001215,3,-45,-112.425907,-11.217285,-8.363398,C,4.360449,13.342121,4.360449
3,40,1,-0.001215,4,-45,-112.425907,-11.217285,-8.363398,C,4.360449,13.342121,4.360449
4,40,1,-0.001215,5,0,-219.285341,-5.566472,0.000000,C,8.982349,6.840403,6.840403


In [21]:
# We can drop the stresses 
StringerStrains = StringerStrains.drop(columns=['Normal_1', 'Normal_2', 'Shear_12','strainX', 'plyTheta'])

# Resort columns to desired order
StringerStrains = StringerStrains[['Elements', 'PlyNumber', 'Loadcase', 'RF_FF', 'RF_IFF', 'Mode', 'RF_strength']]

# We need to analyse element 8 
StringerStrains = StringerStrains[StringerStrains['Elements'] == 60]
#Now we separate the loadcases 
PlyStressesStringerLC1 = StringerStrains[StringerStrains['Loadcase'] == 1]
PlyStressesStringerLC2 = StringerStrains[StringerStrains['Loadcase'] == 2]
PlyStressesStringerLC3 = StringerStrains[StringerStrains['Loadcase'] == 3]
PlyStressesStringerLC1

,Elements,PlyNumber,Loadcase,RF_FF,RF_IFF,Mode,RF_strength
176,60,1,1,13.342124,4.360450,C,4.360450
177,60,2,1,13.342124,4.360450,C,4.360450
178,60,3,1,13.342124,4.360450,C,4.360450
179,60,4,1,13.342124,4.360450,C,4.360450
180,60,5,1,6.840404,8.982352,C,6.840404
181,60,6,1,6.840404,8.982352,C,6.840404
182,60,7,1,269.470560,2.964176,C,2.964176
183,60,8,1,269.470560,2.964176,C,2.964176
184,60,9,1,269.470560,2.964176,C,2.964176
185,60,10,1,269.470560,2.964176,C,2.964176


## Rename Loadcases 

In [22]:
# Loadcase 1
PlyStressesStringerLC1 = PlyStressesStringerLC1.rename(columns={'RF_FF':'R_FF_LC1', 'RF_IFF':'R_IFF_LC1', 'Mode':'Mode_LC1', 'RF_strength':'RF_strength_LC1'})
PlyStressesStringerLC1 = PlyStressesStringerLC1.reset_index()
PlyStressesStringerLC1 = PlyStressesStringerLC1.drop(columns=['index', 'Loadcase'])

# Loadcase 2
PlyStressesStringerLC2 = PlyStressesStringerLC2.rename(columns={'RF_FF':'R_FF_LC2', 'RF_IFF':'R_IFF_LC2', 'Mode':'Mode_LC2', 'RF_strength':'RF_strength_LC2'})
PlyStressesStringerLC2 = PlyStressesStringerLC2.reset_index()
PlyStressesStringerLC2 = PlyStressesStringerLC2.drop(columns=['index', 'Loadcase'])

#Loadcase 3 
PlyStressesStringerLC3 = PlyStressesStringerLC3.rename(columns={'RF_FF':'R_FF_LC3', 'RF_IFF':'R_IFF_LC3', 'Mode':'Mode_LC3', 'RF_strength':'RF_strength_LC3'})
PlyStressesStringerLC3 = PlyStressesStringerLC3.reset_index()
PlyStressesStringerLC3 = PlyStressesStringerLC3.drop(columns=['index', 'Loadcase'])
PlyStressesStringerLC1

,Elements,PlyNumber,R_FF_LC1,R_IFF_LC1,Mode_LC1,RF_strength_LC1
0,60,1,13.342124,4.360450,C,4.360450
1,60,2,13.342124,4.360450,C,4.360450
2,60,3,13.342124,4.360450,C,4.360450
3,60,4,13.342124,4.360450,C,4.360450
4,60,5,6.840404,8.982352,C,6.840404
5,60,6,6.840404,8.982352,C,6.840404
6,60,7,269.470560,2.964176,C,2.964176
7,60,8,269.470560,2.964176,C,2.964176
8,60,9,269.470560,2.964176,C,2.964176
9,60,10,269.470560,2.964176,C,2.964176


## Combine Loadcases 

In [23]:
PlyStressesStringer = PlyStressesStringerLC1.merge(PlyStressesStringerLC2, on=['Elements', 'PlyNumber'], suffixes=('_LC1', '_LC2'))
PlyStressesStringer = PlyStressesStringer.merge(PlyStressesStringerLC3, on=['Elements', 'PlyNumber'])
PlyStressesStringer

,Elements,PlyNumber,R_FF_LC1,R_IFF_LC1,Mode_LC1,RF_strength_LC1,R_FF_LC2,R_IFF_LC2,Mode_LC2,RF_strength_LC2,R_FF_LC3,R_IFF_LC3,Mode_LC3,RF_strength_LC3
0,60,1,13.342124,4.360450,C,4.360450,13.139314,4.294168,C,4.294168,46.805179,15.296789,C,15.296789
1,60,2,13.342124,4.360450,C,4.360450,13.139314,4.294168,C,4.294168,46.805179,15.296789,C,15.296789
2,60,3,13.342124,4.360450,C,4.360450,13.139314,4.294168,C,4.294168,46.805179,15.296789,C,15.296789
3,60,4,13.342124,4.360450,C,4.360450,13.139314,4.294168,C,4.294168,46.805179,15.296789,C,15.296789
4,60,5,6.840404,8.982352,C,6.840404,6.736425,8.845813,C,6.736425,23.996655,31.510769,C,23.996655
5,60,6,6.840404,8.982352,C,6.840404,6.736425,8.845813,C,6.736425,23.996655,31.510769,C,23.996655
6,60,7,269.470560,2.964176,C,2.964176,265.374403,2.919118,C,2.919118,945.323068,10.398554,C,10.398554
7,60,8,269.470560,2.964176,C,2.964176,265.374403,2.919118,C,2.919118,945.323068,10.398554,C,10.398554
8,60,9,269.470560,2.964176,C,2.964176,265.374403,2.919118,C,2.919118,945.323068,10.398554,C,10.398554
9,60,10,269.470560,2.964176,C,2.964176,265.374403,2.919118,C,2.919118,945.323068,10.398554,C,10.398554


## Output stringer

In [24]:
PlyStressesStringer.to_excel(f'../data/{name}/output/stringerStrength.xlsx')